## Import libraries

In [85]:
import pandas as pd

## Read in excel file to pandas

In [86]:
rule_df = pd.read_excel(
    "../data/raw/RiskClassification_Data_Endpoints_V2.xlsx", sheet_name="RiskRules")

# drop the first coulmn
rule_df = rule_df.iloc[:, 1:]
# change server_location to Amaricas to Americas
rule_df["server_location"] = rule_df["server_location"].replace(
    "Amaricas", "Americas")
# fill the empty values with "None"
rule_df = rule_df.fillna("None")
# remove duplicates
rule_df = rule_df.drop_duplicates()
rule_df

,authentication,security_test_category,security_test_result,server_location,hosting_isp,PII,FII,Risk_Label
0,No Authentication,Injections,Fail,Anywhere,Anyone,None,None,Imminent
5,Some Authentication,Broken Authentication,Fail,Anywhere,Anyone,Yes,Yes,Imminent
10,No Authentication,No Test Performed/Available,None,Anywhere,Anyone,Yes,Yes,High
15,Some Authentication,Broken Authentication,Fail,Anywhere,Anyone,No,No,High
20,Some Authentication,All Tests Performed/Available,Pass,Russia,Anyone,Yes,Yes,High
25,Some Authentication,All Tests Performed/Available,Pass,China,Anyone,Yes,Yes,High
30,Some Authentication,No Test Performed/Available,None,Anywhere,Anyone,No,No,Medium
35,Some Authentication,Injections,Fail,Anywhere,Anyone,Yes,Yes,Medium
40,Some Authentication,Buffer Overflow,Fail,Anywhere,Anyone,Yes,Yes,Medium
45,Some Authentication,XML External Enteties,Fail,Anywhere,Anyone,Yes,Yes,Medium


In [87]:
api_df = pd.read_excel("../data/processed/df_pii.xlsx")
# api_df = api_df[['api_endpoint_id', 'authentication', 'security_test_category',
#                  'security_test_result', 'server_location', 'hosting_isp', 'is_pii', 'is_fii']]
api_df


,api_endpoint_id,api_id,api_vendor_id,request_id,method,category,parameters,usage_base,sample_response,tagset,authentication,security_test_category,security_test_result,server_location,hosting_isp,server_name,response_metadata,hosting_city,is_pii,is_fii
0,1,1,1,23688,get,Environment & Weather,"{""q"": ""Vancouver"", ""mode"": ""JSON"", ""type"": ""ac...",NaN,"{""coord"":{""lon"":-123.1193,""lat"":49.2497},""weat...","cod,temp,icon,speed,name,sunrise,clouds,all,de...",query,NaN,NaN,United States,"DigitalOcean, LLC",openresty,"{""Date"": ""Mon, 23 Aug 2021 19:49:39 GMT"", ""Ser...",New York,True,False
1,2,2,2,23597,get,Information & Science,"{""q"": ""hello world""}",NaN,"{\n ""kind"": ""books#volumes"",\n ""totalItems"":...","averageRating,panelizationSummary,webReaderLin...",query,NaN,NaN,Canada,Google LLC,ESF,"{""Date"": ""Fri, 30 Jul 2021 21:48:43 GMT"", ""Var...",Montreal,True,False
2,2,2,2,23597,get,Information & Science,"{""q"": ""hello world""}",NaN,"{\n ""kind"": ""books#volumes"",\n ""totalItems"":...","averageRating,panelizationSummary,webReaderLin...",query,NaN,NaN,Canada,Google LLC,ESF,"{""Date"": ""Fri, 30 Jul 2021 21:48:43 GMT"", ""Var...",Toronto,True,False
3,2,2,2,23597,get,Information & Science,"{""q"": ""hello world""}",NaN,"{\n ""kind"": ""books#volumes"",\n ""totalItems"":...","averageRating,panelizationSummary,webReaderLin...",query,NaN,NaN,Canada,Google LLC,GSE,"{""Date"": ""Fri, 30 Jul 2021 21:48:43 GMT"", ""Var...",Montreal,True,False
4,2,2,2,23597,get,Information & Science,"{""q"": ""hello world""}",NaN,"{\n ""kind"": ""books#volumes"",\n ""totalItems"":...","averageRating,panelizationSummary,webReaderLin...",query,NaN,NaN,Canada,Google LLC,GSE,"{""Date"": ""Fri, 30 Jul 2021 21:48:43 GMT"", ""Var...",Toronto,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1880,2648,1185,462,9897,get,Work & Personal Life,{},NaN,<!DOCTYPE html>\n<html>\n<head>\n <title>The ...,NaN,none,NaN,NaN,United States,Amazon Technologies Inc.,Cowboy,"{""Via"": ""1.1 vegur"", ""Date"": ""Thu, 13 Aug 2020...",Ashburn,False,False
1881,2649,1186,463,9788,get,Justice & Public Safety,{},free,"[{""id"":""avon-and-somerset"",""name"":""Avon and So...","id,name,name",none,NaN,NaN,Ireland,BellSouth.net Inc.,Unavailable/Obscured,"{""Date"": ""Wed, 29 Jul 2020 01:32:25 GMT"", ""Var...",Dublin,True,False
1882,2650,1186,463,9544,get,Justice & Public Safety,"{""force-name"": ""bedfordshire""}",free,"{""description"":""<p>Bedfordshire Police is dedi...","engagement_methods,telephone,type,name,title,i...",none,NaN,NaN,Ireland,"Amazon.com, Inc.",Unavailable/Obscured,"{""Date"": ""Tue, 14 Jul 2020 04:15:03 GMT"", ""Var...",Dublin,True,False
1883,2651,1186,463,9546,get,Justice & Public Safety,"{""force-name"": ""leicestershire""}",free,"[{""bio"":""<p>Simon Cole QPM took up his positio...","bio,twitter,name,rank,contact_details",none,NaN,NaN,Ireland,Amazon Technologies Inc.,Unavailable/Obscured,"{""Date"": ""Tue, 14 Jul 2020 04:20:40 GMT"", ""Var...",Dublin,True,False


In [88]:
print(f'Unique values in column "method": {list(api_df["method"].unique())}\n')
print(
    f'Unique values in column "category": {list(api_df["category"].unique())}\n')
print(
    f'Unique values in column "usage_base": {list(api_df["usage_base"].unique())}\n')
print(
    f'Unique values in column "server_name": {list(api_df["server_name"].unique())}\n')
print(
    f'Unique values in column "hosting_city": {list(api_df["hosting_city"].unique())}\n')



Unique values in column "method": ['get', 'post', 'delete', 'put']

Unique values in column "category": ['Environment & Weather', 'Information & Science', 'GeoInformatics & Navigation', 'Software & Services', 'Social Media & Technology', 'AI & Data Science', 'Government & Public Services', 'News & Media', 'Finance & Banking', 'Business & Technology', 'Health Science & Medicine', 'Transportation & Automobile', 'Justice & Public Safety', 'Natural Resources & Energy', 'Skills & Career Development', 'Research & Education', 'Sports & Entertainment', 'Sales & Marketing', 'eCommerce & Trade', 'Work & Personal Life', 'None', 'Security & Technology', 'Religion & Spirituality', 'Logistics & Infrastructure']

Unique values in column "usage_base": [nan, 'free', 'commercial', 'free_with_authentication']

Unique values in column "server_name": ['openresty', 'ESF', 'GSE', 'mafe', 'Unavailable/Obscured', nan, 'Apache-Coyote/1.1', 'GitHub.com', 'scaffolding on HTTPServer2', 'Missing/ Obscured', 'tsa_b'

In [89]:
# get row containing High
high_risk_rule_df = rule_df[rule_df["Risk_Label"] == "High"]
high_risk_rule_df


,authentication,security_test_category,security_test_result,server_location,hosting_isp,PII,FII,Risk_Label
10,No Authentication,No Test Performed/Available,None,Anywhere,Anyone,Yes,Yes,High
15,Some Authentication,Broken Authentication,Fail,Anywhere,Anyone,No,No,High
20,Some Authentication,All Tests Performed/Available,Pass,Russia,Anyone,Yes,Yes,High
25,Some Authentication,All Tests Performed/Available,Pass,China,Anyone,Yes,Yes,High


In [90]:
# drop Risk_Label column
high_risk_rule_df = high_risk_rule_df.drop(columns=["Risk_Label"])
high_risk_rule_df


,authentication,security_test_category,security_test_result,server_location,hosting_isp,PII,FII
10,No Authentication,No Test Performed/Available,None,Anywhere,Anyone,Yes,Yes
15,Some Authentication,Broken Authentication,Fail,Anywhere,Anyone,No,No
20,Some Authentication,All Tests Performed/Available,Pass,Russia,Anyone,Yes,Yes
25,Some Authentication,All Tests Performed/Available,Pass,China,Anyone,Yes,Yes


In [91]:
from numpy import nan
rules_dict = {}
rules_dict["Some Authentication"] = ["query", "OAuth2",
                                    "header", "BasicAuth", "body", "OAuth1", "path"]
rules_dict["No Authentication"] = ["None"]

rules_dict["All Tests Performed/Available"] = ['XML External Entities',
                                                      'Cross-Site Scripting', 'Insecure Deserialization', 'SQL Injection']
rules_dict["Broken Authentication"] = ['Broken Authentication']
rules_dict["No Test Performed/Available"] = ['None']

rules_dict["None"] = [nan]
rules_dict["Fail"] = [1]
rules_dict["Pass"] = [0]

rules_dict["Russia"] = ["Russia"]
rules_dict["China"] = ["China"]
rules_dict["Anywhere"] = ['United States', 'Canada', 'Germany', 'Netherlands',
                             'United Kingdom', 'Ireland', 'Australia', 'India', 'Luxembourg',
                             'Sweden', 'France', 'Spain', 'Bangladesh', 'Singapore',
                             'Czechia']
rules_dict["Anyone"] = ['United States', 'Canada', 'Germany', 'Netherlands',
                        'United Kingdom', 'Ireland', 'Australia', 'India', 'Luxembourg',
                        'Sweden', 'France', 'Spain', 'Bangladesh', 'Singapore', 'Russia',
                        'Czechia']
rules_dict["Yes"] = [True]
rules_dict["No"] = [False]
rules_dict["method"] = ['get', 'post', 'delete', 'put']
rules_dict["category"] = ['Environment & Weather', 'Information & Science',
                          'GeoInformatics & Navigation', 'Software & Services',
                          'Social Media & Technology', 'AI & Data Science',
                          'Government & Public Services', 'News & Media', 'Finance & Banking',
                          'Business & Technology', 'Health Science & Medicine',
                          'Transportation & Automobile', 'Justice & Public Safety',
                          'Natural Resources & Energy', 'Skills & Career Development',
                          'Research & Education', 'Sports & Entertainment', 'Sales & Marketing',
                          'eCommerce & Trade', 'Work & Personal Life','Security & Technology',
                          'Religion & Spirituality', 'Logistics & Infrastructure']
rules_dict["usage_base"] = ['free', 'commercial', 'free_with_authentication']
rules_dict["server_name"] = ['openresty', 'ESF', 'GSE', 'mafe', 'Unavailable/Obscured', 'Apache-Coyote/1.1', 'GitHub.com', 'scaffolding on HTTPServer2', 'Missing/ Obscured', 'tsa_b', 'Microsoft-IIS/10.0', 'nginx', 'Apache', 'Google Frontend', 'snooserv', 'cloudflare', 'mw1313.eqiad.wmnet', 'mw1377.eqiad.wmnet', 'Apache/2.4.41 (Ubuntu)', 'Concierge', 'Microsoft-IIS/8.5', 'Apache/2.4.18 (Ubuntu)', 'Apache/2.4.7 (Ubuntu)', 'Apache/2.4.6 (Red Hat Enterprise Linux) OpenSSL/1.0.2k-fips', 'Europa', 'Apache/2.2.27 (Unix) PHP/5.5.11', 'Microsoft-IIS/8.0', 'nginx/1.13.1', 'Apache/2.4.6 (Red Hat Enterprise Linux) OpenSSL/1.0.2k-fips mod_wsgi/3.4 Python/2.7.5', 'nginx/1.1.19', 'nginx/1.4.6 (Ubuntu)', 'CloudFront', 'akka-http/10.0.5', 'webserver', 'nginx/1.10.3', 'awselb/2.0', 'api-gateway/1.9.3.1',
                             'Apache/2.4.38 (Debian)', 'thin', 'nginx/1.17.7', 'Apache/2.4.29', 'Tengine', 'Apache/2.4.41 (Amazon) PHP/7.0.33', 'envoy', 'Apache/2.4.29 (Ubuntu)', 'Cowboy', 'akka-http/10.0.15', 'RapidAPI-1.1.0', 'Apache/2.4.38 (Ubuntu)', 'Neutrino-API', 'Python/3.6 aiohttp/3.6.2', 'gunicorn/19.4.5', 'istio-envoy', 'Cloudmersive Server', 'nginx/1.14.0 (Ubuntu)', 'Apache/2.2.15 (CentOS)', 'nginx/1.16.1', 'Apache/2.2.9 (Ubuntu) PHP/5.2.6-2ubuntu4.6 with Suhosin-Patch mod_ssl/2.2.9 OpenSSL/0.9.8g mod_wsgi/4.5.7 Python/2.6', 'openresty/1.13.6.2', 'nginx/1.8.0 + Phusion Passenger 5.0.23', 'Apache/2.4.33 (Amazon) mod_wsgi/3.5 Python/2.7.14 PHP/7.0.30 mod_perl/2.0.10 Perl/v5.16.3', 'nginx/1.17.10', 'nginx/1.17.8', 'nginx/1.17.9', 'nginx/1.6.2', 'nginx/1.12.2', 'nginx/1.15.9', 'Caddy']
rules_dict["hosting_city"] = ['New York', 'Montreal', 'Toronto', 'Dallas', 'Mountain View', 'QuÃ©bec', 'Champaign', 'Indianapolis', 'Portland', 'Seattle', 'Atlanta', 'Newark', 'San Francisco', 'Ashburn', 'Walldorf', "'s-Hertogenbosch", 'Chicago', 'Washington', 'Boston', 'Miami', 'Los Angeles', 'Godmanchester', 'Redmond', 'Amsterdam-Zuidoost', 'Dublin', 'Alexandria', 'Lacey', 'Chehalis', 'Sydney', 'Berkeley', 'San Jose', 'Durango', 'Bokaro Steel City', 'Dumka', 'Ranchi', 'Frankfurt am Main', 'Luxembourg',
                              'Calgary', 'Victoria', 'Vancouver', 'Charlotte', 'Redwood Shores', 'Skurup', 'San Antonio', 'Irmo', 'Rockville', 'Nuremberg', 'Islip', 'Roubaix', 'Falkenstein', 'Seville', 'Township of Piscataway', 'London', 'Saint-Quentin', 'Eindhoven', 'Phoenix', 'Ogden', 'Quincy', 'Cedar Knolls', 'Paris', 'Brentwood', 'DÃ¼sseldorf', 'Clifton', 'Denver', 'Burnaby', 'Brea', 'Asheville', 'Dhaka', 'Mumbai', 'Singapore', 'Moscow', 'Burlington', 'Nhulunbuy', 'Stamford', 'Burntwood', 'Bengaluru', 'Prague', 'Wood Green', 'North Bergen']
                             


In [92]:
# get columns from api_df
api_df_cols = api_df.columns
api_df_cols


Index(['api_endpoint_id', 'api_id', 'api_vendor_id', 'request_id', 'method',
       'category', 'parameters', 'usage_base', 'sample_response', 'tagset',
       'authentication', 'security_test_category', 'security_test_result',
       'server_location', 'hosting_isp', 'server_name', 'response_metadata',
       'hosting_city', 'is_pii', 'is_fii'],
      dtype='object')

In [93]:
# create dataframe with columns from api_df_cols
new_df = pd.DataFrame(columns =  api_df_cols)
new_df


,api_endpoint_id,api_id,api_vendor_id,request_id,method,category,parameters,usage_base,sample_response,tagset,authentication,security_test_category,security_test_result,server_location,hosting_isp,server_name,response_metadata,hosting_city,is_pii,is_fii


In [94]:
import random
starting_id = 3000
# create 100 rows
for i in range(300):
    # generate random values for each column
    starting_id += 1
    new_df.loc[i, "api_endpoint_id"] = starting_id

    # pick a random value for method
    new_df.loc[i, "method"] = random.choice(rules_dict["method"])

    # pick a random value for category
    new_df.loc[i, "category"] = random.choice(rules_dict["category"])

    # pick a random value for usage_base
    new_df.loc[i, "usage_base"] = random.choice(rules_dict["usage_base"])

    # pick a random value for server_name
    new_df.loc[i, "server_name"] = random.choice(rules_dict["server_name"])

    # pick a random value for hosting_city
    new_df.loc[i, "hosting_city"] = random.choice(rules_dict["hosting_city"])

    # pick a random row from high_risk_rule_df
    random_row = high_risk_rule_df.sample(1)

    #authentication
    auth_rule_pick = random_row['authentication'].to_string(index=False)
    auth_rule_value_list = rules_dict[auth_rule_pick]
    # pick random value from auth_rule_value_list
    auth_rule_value = random.choice(auth_rule_value_list)
    new_df.loc[i, "authentication"] = auth_rule_value

    #security_test_category
    security_test_category_rule_pick = random_row['security_test_category'].to_string(index=False)
    security_test_category_rule_value_list = rules_dict[security_test_category_rule_pick]
    security_test_category_rule_value = random.choice(security_test_category_rule_value_list)
    new_df.loc[i, "security_test_category"] = security_test_category_rule_value

    #security_test_result
    security_test_result_rule_pick = random_row['security_test_result'].to_string(index=False)
    security_test_result_rule_value_list = rules_dict[security_test_result_rule_pick]
    security_test_result_rule_value = random.choice(security_test_result_rule_value_list)
    new_df.loc[i, "security_test_result"] = security_test_result_rule_value

    #server_location
    server_location_rule_pick = random_row['server_location'].to_string(index=False)
    server_location_rule_value_list = rules_dict[server_location_rule_pick]
    server_location_rule_value = random.choice(server_location_rule_value_list)
    new_df.loc[i, "server_location"] = server_location_rule_value

    #hosting_isp
    hosting_isp_rule_pick = random_row['hosting_isp'].to_string(index=False)
    hosting_isp_rule_value_list = rules_dict[hosting_isp_rule_pick]
    hosting_isp_rule_value = random.choice(hosting_isp_rule_value_list)
    new_df.loc[i, "hosting_isp"] = hosting_isp_rule_value
    

    #is_pii
    is_pii_rule_pick = random_row['PII'].to_string(index=False)
    is_pii_rule_value_list = rules_dict[is_pii_rule_pick]
    is_pii_rule_value = random.choice(is_pii_rule_value_list)
    new_df.loc[i, "is_pii"] = is_pii_rule_value

    #is_fii
    is_fii_rule_pick = random_row['FII'].to_string(index=False)
    is_fii_rule_value_list = rules_dict[is_fii_rule_pick]
    is_fii_rule_value = random.choice(is_fii_rule_value_list)
    new_df.loc[i, "is_fii"] = is_fii_rule_value




new_df 


,api_endpoint_id,api_id,api_vendor_id,request_id,method,category,parameters,usage_base,sample_response,tagset,authentication,security_test_category,security_test_result,server_location,hosting_isp,server_name,response_metadata,hosting_city,is_pii,is_fii
0,3001,NaN,NaN,NaN,put,Sports & Entertainment,NaN,free,NaN,NaN,path,XML External Entities,0,Russia,Russia,nginx/1.4.6 (Ubuntu),NaN,Calgary,True,True
1,3002,NaN,NaN,NaN,delete,Government & Public Services,NaN,free_with_authentication,NaN,NaN,OAuth2,Broken Authentication,1,Netherlands,Singapore,Europa,NaN,Moscow,False,False
2,3003,NaN,NaN,NaN,get,Environment & Weather,NaN,free_with_authentication,NaN,NaN,None,None,NaN,France,Ireland,awselb/2.0,NaN,Portland,True,True
3,3004,NaN,NaN,NaN,post,Skills & Career Development,NaN,commercial,NaN,NaN,None,None,NaN,Czechia,Luxembourg,ESF,NaN,Portland,True,True
4,3005,NaN,NaN,NaN,post,eCommerce & Trade,NaN,free_with_authentication,NaN,NaN,None,None,NaN,Ireland,Australia,openresty/1.13.6.2,NaN,Washington,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,3296,NaN,NaN,NaN,post,Information & Science,NaN,free,NaN,NaN,path,Insecure Deserialization,0,China,Ireland,GSE,NaN,Lacey,True,True
296,3297,NaN,NaN,NaN,post,Information & Science,NaN,commercial,NaN,NaN,header,SQL Injection,0,China,Australia,api-gateway/1.9.3.1,NaN,Frankfurt am Main,True,True
297,3298,NaN,NaN,NaN,put,Software & Services,NaN,free_with_authentication,NaN,NaN,BasicAuth,Cross-Site Scripting,0,China,Canada,CloudFront,NaN,'s-Hertogenbosch,True,True
298,3299,NaN,NaN,NaN,put,Software & Services,NaN,free,NaN,NaN,None,None,NaN,Czechia,Spain,ESF,NaN,Indianapolis,True,True


In [95]:
# merge new_df with api_df
merged_df = pd.concat([api_df, new_df], axis=0, ignore_index=True)
merged_df


,api_endpoint_id,api_id,api_vendor_id,request_id,method,category,parameters,usage_base,sample_response,tagset,authentication,security_test_category,security_test_result,server_location,hosting_isp,server_name,response_metadata,hosting_city,is_pii,is_fii
0,1,1,1,23688,get,Environment & Weather,"{""q"": ""Vancouver"", ""mode"": ""JSON"", ""type"": ""ac...",NaN,"{""coord"":{""lon"":-123.1193,""lat"":49.2497},""weat...","cod,temp,icon,speed,name,sunrise,clouds,all,de...",query,NaN,NaN,United States,"DigitalOcean, LLC",openresty,"{""Date"": ""Mon, 23 Aug 2021 19:49:39 GMT"", ""Ser...",New York,True,False
1,2,2,2,23597,get,Information & Science,"{""q"": ""hello world""}",NaN,"{\n ""kind"": ""books#volumes"",\n ""totalItems"":...","averageRating,panelizationSummary,webReaderLin...",query,NaN,NaN,Canada,Google LLC,ESF,"{""Date"": ""Fri, 30 Jul 2021 21:48:43 GMT"", ""Var...",Montreal,True,False
2,2,2,2,23597,get,Information & Science,"{""q"": ""hello world""}",NaN,"{\n ""kind"": ""books#volumes"",\n ""totalItems"":...","averageRating,panelizationSummary,webReaderLin...",query,NaN,NaN,Canada,Google LLC,ESF,"{""Date"": ""Fri, 30 Jul 2021 21:48:43 GMT"", ""Var...",Toronto,True,False
3,2,2,2,23597,get,Information & Science,"{""q"": ""hello world""}",NaN,"{\n ""kind"": ""books#volumes"",\n ""totalItems"":...","averageRating,panelizationSummary,webReaderLin...",query,NaN,NaN,Canada,Google LLC,GSE,"{""Date"": ""Fri, 30 Jul 2021 21:48:43 GMT"", ""Var...",Montreal,True,False
4,2,2,2,23597,get,Information & Science,"{""q"": ""hello world""}",NaN,"{\n ""kind"": ""books#volumes"",\n ""totalItems"":...","averageRating,panelizationSummary,webReaderLin...",query,NaN,NaN,Canada,Google LLC,GSE,"{""Date"": ""Fri, 30 Jul 2021 21:48:43 GMT"", ""Var...",Toronto,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2180,3296,NaN,NaN,NaN,post,Information & Science,NaN,free,NaN,NaN,path,Insecure Deserialization,0,China,Ireland,GSE,NaN,Lacey,True,True
2181,3297,NaN,NaN,NaN,post,Information & Science,NaN,commercial,NaN,NaN,header,SQL Injection,0,China,Australia,api-gateway/1.9.3.1,NaN,Frankfurt am Main,True,True
2182,3298,NaN,NaN,NaN,put,Software & Services,NaN,free_with_authentication,NaN,NaN,BasicAuth,Cross-Site Scripting,0,China,Canada,CloudFront,NaN,'s-Hertogenbosch,True,True
2183,3299,NaN,NaN,NaN,put,Software & Services,NaN,free,NaN,NaN,None,None,NaN,Czechia,Spain,ESF,NaN,Indianapolis,True,True


In [96]:
# write merged_df to excel
merged_df.to_excel(
    "../data/processed/df_generated_high_risk.xlsx", index=False)
